In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import backend as K
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Custom Focal Loss Function
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_true = tf.convert_to_tensor(y_true, tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, tf.float32)
        alpha_t = y_true * alpha + (K.ones_like(y_true) - y_true) * (1 - alpha)
        p_t = y_true * y_pred + (K.ones_like(y_true) - y_true) * (1 - y_pred)
        focal_loss = -alpha_t * K.pow((1 - p_t), gamma) * K.log(p_t + K.epsilon())
        return K.mean(focal_loss)
    return focal_loss_fixed

# Paths
train_dir = "D:/DATASET/CNN/ballooning/train"
val_dir = "D:/DATASET/CNN/ballooning/test"

In [2]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    channel_shift_range=50,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Data Generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='binary'
)


Found 7973 images belonging to 2 classes.
Found 381 images belonging to 2 classes.


In [3]:
# Compute Class Weights
y_train = train_generator.classes  # Get true labels
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights_dict = dict(enumerate(class_weights))
print(f"Class weights: {class_weights_dict}")


Class weights: {0: 3.1315789473684212, 1: 0.595}


In [4]:
# Initialize ResNet50 Base Model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
for layer in base_model.layers[:-10]:  # Fine-tune last 10 layers
    layer.trainable = False

# Add Custom Layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)  # Adding Batch Normalization
x = Dense(256, activation='relu')(x)  # Additional Dense Layer
x = BatchNormalization()(x)  # Batch Normalization after Dense Layer
predictions = Dense(1, activation='sigmoid')(x)  # Binary classification

# Final Model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile Model with Focal Loss
model.compile(
    optimizer=Adam(learning_rate=0.00001),
    loss=focal_loss(gamma=2., alpha=0.25),
    metrics=['accuracy']
)

In [5]:
# Callbacks: Model Checkpoint, Early Stopping, and Learning Rate Scheduler
checkpoint = ModelCheckpoint(
    'D:/DATASET/CNN/ballooning/best_model.h5',  # Save best model
    monitor='val_loss', 
    save_best_only=True, 
    mode='min', 
    verbose=1
)

early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=5,  # Stop after 5 epochs with no improvement
    mode='min', 
    verbose=1
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5,  # Reduce LR by 50%
    patience=3,  # Wait for 3 epochs with no improvement
    min_lr=1e-7,  # Minimum LR value
    verbose=1
)


In [6]:
# Train the Model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    class_weight=class_weights_dict,
    callbacks=[checkpoint, early_stopping, lr_scheduler]
)

Epoch 1/10
 46/250 [====>.........................] - ETA: 10:41 - loss: 0.0929 - accuracy: 0.6083

KeyboardInterrupt: 

In [ ]:
# Evaluate the model
eval_result = model.evaluate(val_generator)
print(f"Validation Loss: {eval_result[0]}, Validation Accuracy: {eval_result[1]}")

# Plot Training and Validation Loss/Accuracy
def plot_history(history):
    # Plot accuracy
    plt.plot(history.history['accuracy'], label='train accuracy')
    plt.plot(history.history['val_accuracy'], label='val accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()
    
    # Plot loss
    plt.plot(history.history['loss'], label='train loss')
    plt.plot(history.history['val_loss'], label='val loss')
    plt.title('Model Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

plot_history(history)